# 코로나19 통합 감염 현황

In [92]:
# 공공데이터포털 오픈API 사용
# 공공데이터활용지원센터_보건복지부 코로나19 감염 현황

## 1. API 사용법 익히기

In [93]:
# 라이브러리 설치
!pip install pandas

In [94]:
# 필요한 라이브러리 Import
import requests
from bs4 import BeautifulSoup as bs
from urllib import parse
import pandas as pd

In [95]:
# serviceKey = 'API 일반인증키' (공공데이터포털에서 발급받은 서비스키)
serviceKey = 'jzKP7EwNPdYnAExeAldDB%2FzFTAu3KjXlmGTs04CgNKjULRXTka08HFWvSU0MYLyW87Wmz5hm%2FxxL9drGB9h5sQ%3D%3D'
params = {'ServiceKey':parse.unquote(serviceKey), # 서비스키(필수)
          'startCreateDt':20201201, # 데이터 생성일 시작범위(선택)
          'endCreateDt':20201202} # 데이터 생성일 종료범위(선택)

In [96]:
# 서비스URL
url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'

In [97]:
res = requests.get(url, params=params)
soup = bs(res.text, 'lxml')

In [98]:
# 결과 출력
print(soup)

<?xml version="1.0" encoding="UTF-8" standalone="yes"?><html><body><response><header><resultcode>00</resultcode><resultmsg>NORMAL SERVICE.</resultmsg></header><items><item><accdefrate>1.1559220912</accdefrate><accexamcnt>3106968</accexamcnt><accexamcompcnt>3041814</accexamcompcnt><carecnt>6570</carecnt><clearcnt>28065</clearcnt><createdt>2020-12-02 09:35:32.913</createdt><deathcnt>526</deathcnt><decidecnt>35161</decidecnt><examcnt>65154</examcnt><resutlnegcnt>3006653</resutlnegcnt><seq>340</seq><statedt>20201202</statedt><statetime>00:00</statetime><updatedt>2021-04-20 16:39:02.139</updatedt></item><item><accdefrate>1.1476918806</accdefrate><accexamcnt>3083995</accexamcnt><accexamcompcnt>3019103</accexamcompcnt><carecnt>6239</carecnt><clearcnt>27885</clearcnt><createdt>2020-12-01 09:36:29.45</createdt><deathcnt>526</deathcnt><decidecnt>34650</decidecnt><examcnt>64892</examcnt><resutlnegcnt>2984453</resutlnegcnt><seq>339</seq><statedt>20201201</statedt><statetime>00:00</statetime><updat

## 2. xml 포맷을 Pandas DataFrame으로 변환하기

In [99]:
# 필요한 데이터는 <item></item> "item"태그 내부에 있다.
# BeautifulSoup을 사용해 item 태그만 리스트 형태로 가져오기.

In [100]:
items = soup.find_all('item')
print(items)

[<item><accdefrate>1.1559220912</accdefrate><accexamcnt>3106968</accexamcnt><accexamcompcnt>3041814</accexamcompcnt><carecnt>6570</carecnt><clearcnt>28065</clearcnt><createdt>2020-12-02 09:35:32.913</createdt><deathcnt>526</deathcnt><decidecnt>35161</decidecnt><examcnt>65154</examcnt><resutlnegcnt>3006653</resutlnegcnt><seq>340</seq><statedt>20201202</statedt><statetime>00:00</statetime><updatedt>2021-04-20 16:39:02.139</updatedt></item>, <item><accdefrate>1.1476918806</accdefrate><accexamcnt>3083995</accexamcnt><accexamcompcnt>3019103</accexamcompcnt><carecnt>6239</carecnt><clearcnt>27885</clearcnt><createdt>2020-12-01 09:36:29.45</createdt><deathcnt>526</deathcnt><decidecnt>34650</decidecnt><examcnt>64892</examcnt><resutlnegcnt>2984453</resutlnegcnt><seq>339</seq><statedt>20201201</statedt><statetime>00:00</statetime><updatedt>2021-04-20 16:39:24.094</updatedt></item>]


In [101]:
# 리스트 원소 확인
for x in items:
    print(x)
    print()

<item><accdefrate>1.1559220912</accdefrate><accexamcnt>3106968</accexamcnt><accexamcompcnt>3041814</accexamcompcnt><carecnt>6570</carecnt><clearcnt>28065</clearcnt><createdt>2020-12-02 09:35:32.913</createdt><deathcnt>526</deathcnt><decidecnt>35161</decidecnt><examcnt>65154</examcnt><resutlnegcnt>3006653</resutlnegcnt><seq>340</seq><statedt>20201202</statedt><statetime>00:00</statetime><updatedt>2021-04-20 16:39:02.139</updatedt></item>

<item><accdefrate>1.1476918806</accdefrate><accexamcnt>3083995</accexamcnt><accexamcompcnt>3019103</accexamcompcnt><carecnt>6239</carecnt><clearcnt>27885</clearcnt><createdt>2020-12-01 09:36:29.45</createdt><deathcnt>526</deathcnt><decidecnt>34650</decidecnt><examcnt>64892</examcnt><resutlnegcnt>2984453</resutlnegcnt><seq>339</seq><statedt>20201201</statedt><statetime>00:00</statetime><updatedt>2021-04-20 16:39:24.094</updatedt></item>



### API로 받은 xml 형태의 데이터를 보다 쉽게 다루기 위해 'DataFrame'으로 변환하는 과정

In [102]:
# xml을 dictionary로 변환
response = {'seq':'고유번호',
            'statedt':'기준일',
            'statetime':'기준시간',
            'decidecnt':'확진자',
            'clearcnt':'격리해제',
            'examcnt':'검사진행',
            'deathcnt':'사망자',
            'carecnt':'치료중',
            'resutlnegcnt':'음성',
            'accexamcnt':'누적검사',
            'accexamcompcnt':'누적검사완료',
            'accdefrate':'누적확진률',
            'createdt':'등록일시',
            'updatedt':'수정'}

#### 방법1. xml 태그

In [103]:
# xml을 dictionary로 바꿔 리스트 원소로 추가
lst = []
for y in items:
    l = {}
    for x in y:
        l[x.name] = x.text
    lst.append(l)

lst

[{'accdefrate': '1.1559220912',
  'accexamcnt': '3106968',
  'accexamcompcnt': '3041814',
  'carecnt': '6570',
  'clearcnt': '28065',
  'createdt': '2020-12-02 09:35:32.913',
  'deathcnt': '526',
  'decidecnt': '35161',
  'examcnt': '65154',
  'resutlnegcnt': '3006653',
  'seq': '340',
  'statedt': '20201202',
  'statetime': '00:00',
  'updatedt': '2021-04-20 16:39:02.139'},
 {'accdefrate': '1.1476918806',
  'accexamcnt': '3083995',
  'accexamcompcnt': '3019103',
  'carecnt': '6239',
  'clearcnt': '27885',
  'createdt': '2020-12-01 09:36:29.45',
  'deathcnt': '526',
  'decidecnt': '34650',
  'examcnt': '64892',
  'resutlnegcnt': '2984453',
  'seq': '339',
  'statedt': '20201201',
  'statetime': '00:00',
  'updatedt': '2021-04-20 16:39:24.094'}]

In [104]:
# list를 dataframe으로 만들기
df = pd.DataFrame(lst)
df.columns = df.columns.map(response)
df

누적확진률     누적검사   누적검사완료   치료중   격리해제                     등록일시  사망자  \
0  1.1559220912  3106968  3041814  6570  28065  2020-12-02 09:35:32.913  526   
1  1.1476918806  3083995  3019103  6239  27885   2020-12-01 09:36:29.45  526   

     확진자   검사진행       음성 고유번호       기준일   기준시간                       수정  
0  35161  65154  3006653  340  20201202  00:00  2021-04-20 16:39:02.139  
1  34650  64892  2984453  339  20201201  00:00  2021-04-20 16:39:24.094

#### 방법2. xmltodict 라이브러리 사용하기

In [105]:
# xmltodict 설치
!pip install xmltodict

In [106]:
import xmltodict
xmltodict.parse(str(items[0]))['item']

OrderedDict([('accdefrate', '1.1559220912'),
             ('accexamcnt', '3106968'),
             ('accexamcompcnt', '3041814'),
             ('carecnt', '6570'),
             ('clearcnt', '28065'),
             ('createdt', '2020-12-02 09:35:32.913'),
             ('deathcnt', '526'),
             ('decidecnt', '35161'),
             ('examcnt', '65154'),
             ('resutlnegcnt', '3006653'),
             ('seq', '340'),
             ('statedt', '20201202'),
             ('statetime', '00:00'),
             ('updatedt', '2021-04-20 16:39:02.139')])

In [107]:
# xml 형태의 문자열을 입력값으로 받는 'xmltodict'은 'OrdereDict'형태로 결과값을 반환하기 때문에 'dict()'를 통해 'dictionary'로 변환
dict(xmltodict.parse(str(items[0]))['item'])

{'accdefrate': '1.1559220912',
 'accexamcnt': '3106968',
 'accexamcompcnt': '3041814',
 'carecnt': '6570',
 'clearcnt': '28065',
 'createdt': '2020-12-02 09:35:32.913',
 'deathcnt': '526',
 'decidecnt': '35161',
 'examcnt': '65154',
 'resutlnegcnt': '3006653',
 'seq': '340',
 'statedt': '20201202',
 'statetime': '00:00',
 'updatedt': '2021-04-20 16:39:02.139'}

In [108]:
# 'xml'을 'dictionary'로 변환하여 'DataFrame'으로 만들 수 있다.
dic2 = dict(covid19=[])
for item in items:
    dic2['covid19'].append(dict(xmltodict.parse(str(item))['item']))
    
# 'DataFrame'으로 변환
df = pd.DataFrame(dic2['covid19'])
df.columns = df.columns.map(response) # column명에 map함수를 적용하여 한글로 변환
df

누적확진률     누적검사   누적검사완료   치료중   격리해제                     등록일시  사망자  \
0  1.1559220912  3106968  3041814  6570  28065  2020-12-02 09:35:32.913  526   
1  1.1476918806  3083995  3019103  6239  27885   2020-12-01 09:36:29.45  526   

     확진자   검사진행       음성 고유번호       기준일   기준시간                       수정  
0  35161  65154  3006653  340  20201202  00:00  2021-04-20 16:39:02.139  
1  34650  64892  2984453  339  20201201  00:00  2021-04-20 16:39:24.094

## 3. Pandas DataFrame 컨트롤

In [109]:
# API로 최근 일주일간의 데이터 재요청하여 'DataFrame'으로 변환

# 1. API로 데이터 불러오기
import requests
from bs4 import BeautifulSoup as bs
from urllib import parse
import pandas as pd

import datetime
today = datetime.datetime.now() # datetime 라이브러리를 통해 현재 날짜 today 생성
weeks = today-datetime.timedelta(days=6)

serviceKey = 'jzKP7EwNPdYnAExeAldDB%2FzFTAu3KjXlmGTs04CgNKjULRXTka08HFWvSU0MYLyW87Wmz5hm%2FxxL9drGB9h5sQ%3D%3D'
params = {'ServiceKey':parse.unquote(serviceKey),
          'startCreateDt': weeks.strftime('%Y%m%d'),
          'endCreateDt': today.strftime('%Y%m%d')}

url = 'http://openapi.data.go.kr/openapi/service/rest/Covid19/getCovid19InfStateJson'

res = requests.get(url, params=params)
soup = bs(res.text, 'lxml')

# 2. xml을 DataFrame으로 변환하기
items = soup.find_all('item')

response = {'seq':'고유번호',
            'statedt':'기준일',
            'statetime':'기준시간',
            'decidecnt':'확진자',
            'clearcnt':'격리해제',
            'examcnt':'검사진행',
            'deathcnt':'사망자',
            'carecnt':'치료중',
            'resutlnegcnt':'음성',
            'accexamcnt':'누적검사',
            'accexamcompcnt':'누적검사완료',
            'accdefrate':'누적확진률',
            'createdt':'등록일시',
            'updatedt':'수정'}

lst = []
for y in items:
    l = {}
    for x in y:
        l[x.name] = x.text
    lst.append(l)

df = pd.DataFrame(lst)
df.columns = df.columns.map(response)
df

누적확진률     누적검사   누적검사완료   치료중    격리해제                     등록일시  \
0  1.4541964496  9747614  9621121  7572  130381  2021-05-30 09:24:39.384   
1  1.4509249463  9733591  9609801  7741  129739  2021-05-29 09:45:26.765   
2  1.4499377008  9702456  9579584  8193  128759  2021-05-28 09:34:29.553   
3  1.4493399059  9665693  9543034  8188  128180  2021-05-27 09:39:43.099   
4  1.4479226274  9629450  9508381  8160  127574  2021-05-26 09:46:42.628   
5  1.4455504748  9592727  9475629  8044  126993  2021-05-25 09:33:39.762   
6  1.4466269101  9553613  9432909  8098  126427  2021-05-24 09:42:11.785   

    사망자     확진자    검사진행       음성 고유번호       기준일   기준시간  \
0  1957  139910  126493  9481211  527  20210530  00:00   
1  1951  139431  123790  9470370  526  20210529  00:00   
2  1946  138898  122872  9440686  525  20210528  00:00   
3  1943  138311  122659  9404723  524  20210527  00:00   
4  1940  137674  121069  9370707  523  20210526  00:00   
5  1938  136975  117098  9338654  522  20210525  00:00   
6  1934  136459  120704  9296450  521  20210524  00:00   

                        수정  
0                     null  
1                     null  
2  2021-05-29 09:47:35.466  
3                     null  
4  2021-05-27 09:40:52.468  
5  2021-05-26 17:26:57.787  
6  2021-05-26 17:11:27.494

In [110]:
data = df.copy()
data

누적확진률     누적검사   누적검사완료   치료중    격리해제                     등록일시  \
0  1.4541964496  9747614  9621121  7572  130381  2021-05-30 09:24:39.384   
1  1.4509249463  9733591  9609801  7741  129739  2021-05-29 09:45:26.765   
2  1.4499377008  9702456  9579584  8193  128759  2021-05-28 09:34:29.553   
3  1.4493399059  9665693  9543034  8188  128180  2021-05-27 09:39:43.099   
4  1.4479226274  9629450  9508381  8160  127574  2021-05-26 09:46:42.628   
5  1.4455504748  9592727  9475629  8044  126993  2021-05-25 09:33:39.762   
6  1.4466269101  9553613  9432909  8098  126427  2021-05-24 09:42:11.785   

    사망자     확진자    검사진행       음성 고유번호       기준일   기준시간  \
0  1957  139910  126493  9481211  527  20210530  00:00   
1  1951  139431  123790  9470370  526  20210529  00:00   
2  1946  138898  122872  9440686  525  20210528  00:00   
3  1943  138311  122659  9404723  524  20210527  00:00   
4  1940  137674  121069  9370707  523  20210526  00:00   
5  1938  136975  117098  9338654  522  20210525  00:00   
6  1934  136459  120704  9296450  521  20210524  00:00   

                        수정  
0                     null  
1                     null  
2  2021-05-29 09:47:35.466  
3                     null  
4  2021-05-27 09:40:52.468  
5  2021-05-26 17:26:57.787  
6  2021-05-26 17:11:27.494

### .info() : 데이터 요약 정보

In [111]:
# dataframe의 데이터 타입(Dtype), 행(RangeIndex), 열(Data columns) 크기 등의 정보 출력
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   누적확진률   7 non-null      object
 1   누적검사    7 non-null      object
 2   누적검사완료  7 non-null      object
 3   치료중     7 non-null      object
 4   격리해제    7 non-null      object
 5   등록일시    7 non-null      object
 6   사망자     7 non-null      object
 7   확진자     7 non-null      object
 8   검사진행    7 non-null      object
 9   음성      7 non-null      object
 10  고유번호    7 non-null      object
 11  기준일     7 non-null      object
 12  기준시간    7 non-null      object
 13  수정      7 non-null      object
dtypes: object(14)
memory usage: 912.0+ bytes


### column 문자열 바꾸기

In [112]:
# data type 변경을 위해 "수정"컬럼의 날짜 및 시간정보에 포함된 문자열 'null'을 ''으로 수정
print('적용전')
print(data['수정'])
data['수정'] = data['수정'].str.replace('null','')
print('적용후')
print(data['수정'])

적용전
0                       null
1                       null
2    2021-05-29 09:47:35.466
3                       null
4    2021-05-27 09:40:52.468
5    2021-05-26 17:26:57.787
6    2021-05-26 17:11:27.494
Name: 수정, dtype: object
적용후
0                           
1                           
2    2021-05-29 09:47:35.466
3                           
4    2021-05-27 09:40:52.468
5    2021-05-26 17:26:57.787
6    2021-05-26 17:11:27.494
Name: 수정, dtype: object


### data type 확인 및 변경

In [113]:
for col in data.columns:
    if col in ['등록일시','기준일','수정']: # 날짜 : datetime
        data[col] = data[col].apply(pd.to_datetime)
    elif col == '기준시간': # 시간 : object
        data[col] = data[col].astype('O')
    else : # 나머지 columns : 수치형
        data[col] = data[col].apply(pd.to_numeric)

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   누적확진률   7 non-null      float64       
 1   누적검사    7 non-null      int64         
 2   누적검사완료  7 non-null      int64         
 3   치료중     7 non-null      int64         
 4   격리해제    7 non-null      int64         
 5   등록일시    7 non-null      datetime64[ns]
 6   사망자     7 non-null      int64         
 7   확진자     7 non-null      int64         
 8   검사진행    7 non-null      int64         
 9   음성      7 non-null      int64         
 10  고유번호    7 non-null      int64         
 11  기준일     7 non-null      datetime64[ns]
 12  기준시간    7 non-null      object        
 13  수정      4 non-null      datetime64[ns]
dtypes: datetime64[ns](3), float64(1), int64(9), object(1)
memory usage: 912.0+ bytes


In [114]:
data

누적확진률     누적검사   누적검사완료   치료중    격리해제                    등록일시   사망자  \
0  1.454196  9747614  9621121  7572  130381 2021-05-30 09:24:39.384  1957   
1  1.450925  9733591  9609801  7741  129739 2021-05-29 09:45:26.765  1951   
2  1.449938  9702456  9579584  8193  128759 2021-05-28 09:34:29.553  1946   
3  1.449340  9665693  9543034  8188  128180 2021-05-27 09:39:43.099  1943   
4  1.447923  9629450  9508381  8160  127574 2021-05-26 09:46:42.628  1940   
5  1.445550  9592727  9475629  8044  126993 2021-05-25 09:33:39.762  1938   
6  1.446627  9553613  9432909  8098  126427 2021-05-24 09:42:11.785  1934   

      확진자    검사진행       음성  고유번호        기준일   기준시간                      수정  
0  139910  126493  9481211   527 2021-05-30  00:00                     NaT  
1  139431  123790  9470370   526 2021-05-29  00:00                     NaT  
2  138898  122872  9440686   525 2021-05-28  00:00 2021-05-29 09:47:35.466  
3  138311  122659  9404723   524 2021-05-27  00:00                     NaT  
4  137674  121069  9370707   523 2021-05-26  00:00 2021-05-27 09:40:52.468  
5  136975  117098  9338654   522 2021-05-25  00:00 2021-05-26 17:26:57.787  
6  136459  120704  9296450   521 2021-05-24  00:00 2021-05-26 17:11:27.494

### 중복값 처리

In [115]:
# duplicated를 통해 중복값 확인
data[data.기준일.duplicated(keep=False)]

Empty DataFrame
Columns: [누적확진률, 누적검사, 누적검사완료, 치료중, 격리해제, 등록일시, 사망자, 확진자, 검사진행, 음성, 고유번호, 기준일, 기준시간, 수정]
Index: []

In [116]:
# duplicated(keep='first')는 중복되는 기준일 중 첫번째 데이터(최신 데이터)만 유지하고 나머지는 중복값으로 처리합니다.
# drop 함수를 통해 첫번째 데이터(최신 데이터)만 남기고 나머지 중복 데이터는 삭제합니다.
data = data.drop(index=data[data.기준일.duplicated(keep='first')].index).reset_index(drop=True)
data[data.기준일.duplicated(keep=False)]

Empty DataFrame
Columns: [누적확진률, 누적검사, 누적검사완료, 치료중, 격리해제, 등록일시, 사망자, 확진자, 검사진행, 음성, 고유번호, 기준일, 기준시간, 수정]
Index: []

### 정렬(sorting)

In [117]:
data = data.sort_values('기준일', ascending=False).reset_index(drop=True)
data

누적확진률     누적검사   누적검사완료   치료중    격리해제                    등록일시   사망자  \
0  1.454196  9747614  9621121  7572  130381 2021-05-30 09:24:39.384  1957   
1  1.450925  9733591  9609801  7741  129739 2021-05-29 09:45:26.765  1951   
2  1.449938  9702456  9579584  8193  128759 2021-05-28 09:34:29.553  1946   
3  1.449340  9665693  9543034  8188  128180 2021-05-27 09:39:43.099  1943   
4  1.447923  9629450  9508381  8160  127574 2021-05-26 09:46:42.628  1940   
5  1.445550  9592727  9475629  8044  126993 2021-05-25 09:33:39.762  1938   
6  1.446627  9553613  9432909  8098  126427 2021-05-24 09:42:11.785  1934   

      확진자    검사진행       음성  고유번호        기준일   기준시간                      수정  
0  139910  126493  9481211   527 2021-05-30  00:00                     NaT  
1  139431  123790  9470370   526 2021-05-29  00:00                     NaT  
2  138898  122872  9440686   525 2021-05-28  00:00 2021-05-29 09:47:35.466  
3  138311  122659  9404723   524 2021-05-27  00:00                     NaT  
4  137674  121069  9370707   523 2021-05-26  00:00 2021-05-27 09:40:52.468  
5  136975  117098  9338654   522 2021-05-25  00:00 2021-05-26 17:26:57.787  
6  136459  120704  9296450   521 2021-05-24  00:00 2021-05-26 17:11:27.494

### 일일현황 데이터 추가

In [118]:
# 보건복지부 코로나19현황 오픈API는 확진, 격리, 사망, 치료 등에 대한 데이터를 날짜 기준으로 누적 데이터를 제공한다.
# 따라서 날짜별 일일현황에 대한 데이터를 생성 할 수 있다.

In [119]:
# diff(-1) 끝에서부터 행 단위 차이 계산
daily = data.loc[:,['확진자','사망자','누적검사완료','격리해제']].diff(-1)
daily.columns = ['일확진자','일사망자','일검사완료','일격리해제']
daily

일확진자  일사망자    일검사완료  일격리해제
0  479.0   6.0  11320.0  642.0
1  533.0   5.0  30217.0  980.0
2  587.0   3.0  36550.0  579.0
3  637.0   3.0  34653.0  606.0
4  699.0   2.0  32752.0  581.0
5  516.0   4.0  42720.0  566.0
6    NaN   NaN      NaN    NaN

In [120]:
# concat으로 data, daily 데이터프레임 결합
daily_data = pd.concat([data, daily], axis=1)
daily_data

누적확진률     누적검사   누적검사완료   치료중    격리해제                    등록일시   사망자  \
0  1.454196  9747614  9621121  7572  130381 2021-05-30 09:24:39.384  1957   
1  1.450925  9733591  9609801  7741  129739 2021-05-29 09:45:26.765  1951   
2  1.449938  9702456  9579584  8193  128759 2021-05-28 09:34:29.553  1946   
3  1.449340  9665693  9543034  8188  128180 2021-05-27 09:39:43.099  1943   
4  1.447923  9629450  9508381  8160  127574 2021-05-26 09:46:42.628  1940   
5  1.445550  9592727  9475629  8044  126993 2021-05-25 09:33:39.762  1938   
6  1.446627  9553613  9432909  8098  126427 2021-05-24 09:42:11.785  1934   

      확진자    검사진행       음성  고유번호        기준일   기준시간                      수정  \
0  139910  126493  9481211   527 2021-05-30  00:00                     NaT   
1  139431  123790  9470370   526 2021-05-29  00:00                     NaT   
2  138898  122872  9440686   525 2021-05-28  00:00 2021-05-29 09:47:35.466   
3  138311  122659  9404723   524 2021-05-27  00:00                     NaT   
4  137674  121069  9370707   523 2021-05-26  00:00 2021-05-27 09:40:52.468   
5  136975  117098  9338654   522 2021-05-25  00:00 2021-05-26 17:26:57.787   
6  136459  120704  9296450   521 2021-05-24  00:00 2021-05-26 17:11:27.494   

    일확진자  일사망자    일검사완료  일격리해제  
0  479.0   6.0  11320.0  642.0  
1  533.0   5.0  30217.0  980.0  
2  587.0   3.0  36550.0  579.0  
3  637.0   3.0  34653.0  606.0  
4  699.0   2.0  32752.0  581.0  
5  516.0   4.0  42720.0  566.0  
6    NaN   NaN      NaN    NaN

### 원하는 컬럼만 선택하기

In [121]:
# DataFrame.loc[row label, column label]
ndf = daily_data.loc[:,['기준일','일확진자','일사망자','일검사완료','일격리해제']].dropna()
ndf.loc[:,'일확진자':] = ndf.loc[:,'일확진자':].astype(int)
ndf

기준일  일확진자  일사망자  일검사완료  일격리해제
0 2021-05-30   479     6  11320    642
1 2021-05-29   533     5  30217    980
2 2021-05-28   587     3  36550    579
3 2021-05-27   637     3  34653    606
4 2021-05-26   699     2  32752    581
5 2021-05-25   516     4  42720    566

### CSV 파일로 저장하기

In [122]:
# API로 호출하여 받은 데이터를 파일로 저장하면 매번 API를 호출하지 않아도 된다.
ndf.to_csv('data/corona_Integration_status.csv', index=False, encoding='utf8')

import os
os.listdir('data') # 파일 디렉토리 확인

['corona.csv', 'corona_Integration_status.csv']

## 4. 코로나19 현황 웹사이트 시각화 적용

### 데이터(csv) 불러오기

In [123]:
# plotly 설치 및 임포트
!pip install plotly

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# 저장한 csv파일 불러오기
data = pd.read_csv('data/corona_Integration_status.csv')

# 기준일 data type 날짜로 변환
data.기준일 = data.기준일.apply(pd.to_datetime)
data

기준일  일확진자  일사망자  일검사완료  일격리해제
0 2021-05-30   479     6  11320    642
1 2021-05-29   533     5  30217    980
2 2021-05-28   587     3  36550    579
3 2021-05-27   637     3  34653    606
4 2021-05-26   699     2  32752    581
5 2021-05-25   516     4  42720    566

### 기본 막대 그래프

In [124]:
df = data.copy()
# df = df[df.기준일.df.month == 5].sort_values('기준일') 12월 기준일 오름차순 정렬
df = df.sort_values('기준일')

fig = go.Figure()
# 확진 그래프
fig.add_trace(go.Bar(x=df.기준일.dt.strftime('%m.%d'),
                     y=df.일확진자,
                     text=df.일확진자, # 그래프 위 확진 수 보이게 하기
                     textposition='outside', # text 위치
                     hovertemplate='기준일: %{x} <br>확진: %{y}명',
                     name='확진'
                    )
             )
# 사망 그래프
fig.add_trace(go.Bar(x=df.기준일.dt.strftime('%m.%d'),
                     y=df.일사망자,
                     text=df.일사망자, # 그래프 위 사망 수 보이게 하기
                     textposition='outside', # text 위치
                     hovertemplate='기준일: %{x} <br>사망: {y}명',
                     name='사망'
                    )
             )
# 격리해제 그래프
fig.add_trace(go.Bar(x=df.기준일.dt.strftime('%m.%d'),
                     y=df.일격리해제,
                     text=df.일격리해제, # 그래프 위 격리해제 수 보이게 하기
                     textposition='outside', # text 위치
                     hovertemplate='기준일: %{x} <br>격리해제: {y}명',
                     name='격리해제'
                    )
             )
fig.update_layout(xaxis=dict(type='category'), # x축 모든 날짜 보이게 하기
                  title_text='대한민국 코로나19(COVID-19) 추이 - 2021.05월 4주차' # 그래프 제목
                 )
fig.show()

### 막대 그래프 + Range Slider(날짜 범위 선택)

In [125]:
df = data.copy()
# df = df[(df.기준일.dt.month == 11)|(df.기준일.dt.month == 12)].sort_values('기준일') # 11월~12월 기준일 오름차순 정렬
df = df.sort_values('기준일')

fig = go.Figure()
# 확진 그래프
fig.add_trace(go.Bar(x=df.기준일, # 기본 그래프와 달리 '2020-12-01' 날짜 포맷 그대로 입력
                     y=df.일확진자, 
                     text=df.일확진자, # 그래프 위 확진 수 보이게 하기
                     textposition='outside', # text 위치
                     hovertemplate='기준일: %{x} <br>확진: %{y}명', 
                     name='확진'
                    )
             )
# 사망 그래프
fig.add_trace(go.Bar(x=df.기준일, # 기본 그래프와 달리 '2020-12-01' 날짜 포맷 그대로 입력
                     y=df.일사망자, 
                     text=df.일사망자, # 그래프 위 사망 수 보이게 하기
                     textposition='outside', # text 위치
                     hovertemplate='기준일: %{x} <br>사망: %{y}명',
                     name='사망'
                    )
             )
# 격리해제 그래프
fig.add_trace(go.Bar(x=df.기준일, # 기본 그래프와 달리 '2020-12-01' 날짜 포맷 그대로 입력
                     y=df.일격리해제, 
                     text=df.일격리해제, # 그래프 위 격리해제 수 보이게 하기
                     textposition='outside', # text 위치
                     hovertemplate='기준일: %{x} <br>격리해제: %{y}명',
                     name='격리해제'
                    )
             )
fig.update_layout(xaxis=dict(type='category'),
                  title_text='대한민국 코로나19(COVID-19) 추이 - 2021.05월 4주차' # 그래프 제목
                 )
# Range Slider 추가
fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=7, # day기준 7일 = 1week
                     label="1w",
                     step="day", 
                     stepmode="backward"),
                dict(count=14, # day기준 14일 = 2week
                     label="2w", 
                     step="day",
                     stepmode="backward"),
                dict(count=1, # month기준 1달 = 1month
                     label="1m",
                     step="month",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True
        ),
        type="date"
    )
)

# 기간 초기값 설정
initial_range = ['2021-05-28', '2021-05-30']
fig.update_layout(xaxis=dict(range=initial_range))

fig.show()